## 1. Local Scope

In [ ]:
# Variables defined inside a function are local
def greet():
    message = "Hello, World!"  # Local variable
    print(message)

greet()

# Trying to access outside will cause error
# print(message)  # NameError: name 'message' is not defined

In [ ]:
# Each function call creates new local scope
def counter():
    count = 0  # Fresh variable each call
    count += 1
    print(f"Count: {count}")

counter()  # Count: 1
counter()  # Count: 1 (not 2!)
counter()  # Count: 1

In [ ]:
# Parameters are also local
def process(data):
    data = data.upper()  # Modifies local copy
    print(f"Inside: {data}")

text = "hello"
process(text)
print(f"Outside: {text}")  # Original unchanged

## 2. Global Scope

In [ ]:
# Variables defined at module level are global
app_name = "My Application"  # Global variable
version = "1.0.0"

def display_info():
    # Can READ global variables
    print(f"{app_name} v{version}")

display_info()

In [ ]:
# Trying to modify creates local variable instead
counter = 0  # Global

def increment():
    counter = 1  # Creates LOCAL variable, doesn't modify global
    print(f"Inside: {counter}")

increment()
print(f"Global: {counter}")  # Still 0!

In [ ]:
# Using 'global' keyword to modify
counter = 0

def increment():
    global counter  # Declare we want global variable
    counter += 1
    print(f"Inside: {counter}")

print(f"Before: {counter}")
increment()
increment()
increment()
print(f"After: {counter}")

## 3. Enclosing Scope (Nested Functions)

In [ ]:
# Enclosing scope - outer function's variables
def outer():
    x = 10  # Enclosing scope for inner()
    
    def inner():
        print(f"x from outer: {x}")
    
    inner()

outer()

In [ ]:
# Modifying enclosing variable - need 'nonlocal'
def outer():
    count = 0
    
    def inner():
        nonlocal count  # Access enclosing variable
        count += 1
        print(f"Count: {count}")
    
    inner()
    inner()
    inner()
    print(f"Final: {count}")

outer()

In [ ]:
# Closure - function that remembers enclosing scope
def create_counter(start=0):
    count = start
    
    def increment():
        nonlocal count
        count += 1
        return count
    
    return increment  # Return the function

# Each counter has its own count
counter1 = create_counter()
counter2 = create_counter(100)

print(f"Counter 1: {counter1()}, {counter1()}, {counter1()}")
print(f"Counter 2: {counter2()}, {counter2()}, {counter2()}")

## 4. LEGB Rule

In [ ]:
# Python searches for variables in this order:
# L - Local: Inside current function
# E - Enclosing: In outer functions
# G - Global: At module level
# B - Built-in: Python's built-in names

x = "global"  # Global

def outer():
    x = "enclosing"  # Enclosing
    
    def inner():
        x = "local"  # Local
        print(f"Inner sees: {x}")
    
    inner()
    print(f"Outer sees: {x}")

outer()
print(f"Module sees: {x}")

In [ ]:
# Built-in scope example
# 'len', 'print', 'range' are built-in

print(f"len is built-in: {len([1, 2, 3])}")

# Don't shadow built-ins!
# list = [1, 2, 3]  # BAD! Shadows built-in 'list'
# print(list(range(5)))  # Error! 'list' is now a list, not a function

In [ ]:
# View current scopes
x = "global"

def show_scopes():
    y = "local"
    
    print("Local variables:", list(locals().keys()))
    print("Global variables:", [k for k in globals().keys() if not k.startswith('_')][:5])

show_scopes()

## 5. Common Scope Patterns

In [ ]:
# Pattern 1: Configuration as global constants
CONFIG = {
    "debug": True,
    "max_retries": 3,
    "timeout": 30
}

def connect():
    if CONFIG["debug"]:
        print("Debug mode enabled")
    print(f"Connecting with {CONFIG['max_retries']} retries...")

connect()

In [ ]:
# Pattern 2: Factory function with closure
def create_multiplier(factor):
    """Create a function that multiplies by factor"""
    def multiplier(x):
        return x * factor
    return multiplier

double = create_multiplier(2)
triple = create_multiplier(3)

print(f"Double 5: {double(5)}")
print(f"Triple 5: {triple(5)}")

In [ ]:
# Pattern 3: State management with closure
def create_bank_account(initial_balance):
    """Create account with encapsulated balance"""
    balance = initial_balance
    
    def get_balance():
        return balance
    
    def deposit(amount):
        nonlocal balance
        if amount > 0:
            balance += amount
            return True
        return False
    
    def withdraw(amount):
        nonlocal balance
        if 0 < amount <= balance:
            balance -= amount
            return True
        return False
    
    # Return dictionary of functions
    return {
        "get_balance": get_balance,
        "deposit": deposit,
        "withdraw": withdraw
    }

# Create account
account = create_bank_account(100)

print(f"Initial: ${account['get_balance']}")
account["deposit"](50)
print(f"After deposit: ${account['get_balance']()}")
account["withdraw"](30)
print(f"After withdrawal: ${account['get_balance']()}")

## 6. Best Practices

In [ ]:
# ❌ BAD: Modifying globals
result = 0

def add_bad(a, b):
    global result
    result = a + b  # Side effect!

add_bad(5, 3)
print(f"Bad approach: {result}")

# ✅ GOOD: Return value
def add_good(a, b):
    return a + b  # Pure function

result = add_good(5, 3)
print(f"Good approach: {result}")

In [ ]:
# ❌ BAD: Shadowing built-ins
# list = [1, 2, 3]  # DON'T DO THIS!
# str = "hello"      # DON'T DO THIS!

# ✅ GOOD: Use descriptive names
numbers = [1, 2, 3]
message = "hello"

print(f"Numbers: {numbers}")
print(f"Message: {message}")

In [ ]:
# ✅ GOOD: Pass values as arguments
def process_data(data, multiplier):
    """Process data with given multiplier"""
    return [x * multiplier for x in data]

numbers = [1, 2, 3, 4, 5]
factor = 10
result = process_data(numbers, factor)
print(f"Result: {result}")

## 7. Complete Example: Game State Manager

In [ ]:
def create_game():
    """
    Create a simple game with encapsulated state.
    Uses closures to manage game state.
    """
    # Private state (enclosing scope)
    player_name = ""
    score = 0
    level = 1
    high_scores = []
    
    def start_game(name):
        nonlocal player_name, score, level
        player_name = name
        score = 0
        level = 1
        print(f"\n🎮 Welcome, {player_name}!")
        print(f"   Starting at Level {level}")
    
    def add_points(points):
        nonlocal score, level
        score += points
        print(f"   +{points} points! Total: {score}")
        
        # Level up every 100 points
        new_level = score // 100 + 1
        if new_level > level:
            level = new_level
            print(f"   🎉 Level Up! Now at Level {level}")
    
    def get_status():
        return {
            "player": player_name,
            "score": score,
            "level": level
        }
    
    def end_game():
        nonlocal high_scores
        high_scores.append({"name": player_name, "score": score})
        high_scores.sort(key=lambda x: x["score"], reverse=True)
        high_scores = high_scores[:5]  # Keep top 5
        
        print(f"\n   Game Over, {player_name}!")
        print(f"   Final Score: {score}")
        print(f"   Final Level: {level}")
    
    def show_high_scores():
        print("\n🏆 HIGH SCORES:")
        if not high_scores:
            print("   No scores yet!")
        for i, hs in enumerate(high_scores, 1):
            print(f"   {i}. {hs['name']}: {hs['score']}")
    
    # Return game interface
    return {
        "start": start_game,
        "score": add_points,
        "status": get_status,
        "end": end_game,
        "high_scores": show_high_scores
    }

# Create and play game
game = create_game()

# Player 1
game["start"]("Alice")
game["score"](50)
game["score"](75)
game["score"](30)
game["end"]()

# Player 2
game["start"]("Bob")
game["score"](100)
game["score"](150)
game["end"]()

# Show rankings
game["high_scores"]()

## Summary

### LEGB Rule:

| Scope | Description | Example |
|-------|-------------|----------|
| Local | Inside current function | `def f(): x = 1` |
| Enclosing | In outer function | Nested functions |
| Global | Module level | Top of file |
| Built-in | Python's names | `len`, `print` |

### Keywords:

| Keyword | Purpose |
|---------|----------|
| `global` | Modify global variable |
| `nonlocal` | Modify enclosing variable |

### Best Practices:
1. Avoid modifying globals
2. Don't shadow built-ins
3. Pass data as arguments
4. Return values instead of modifying state
5. Use closures for encapsulation

### Next Lesson: Recursion